In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import special as sp
from scipy import stats
from enum import Enum

import random

In [2]:
class Environment(Enum):
  FREE_SPACE = 0,
  URBAN_AREA_CELLULAR_RADIO = 1,
  SHADOWED_URBAN_CELLULAR_RADIO = 2,
  IN_BUILDING_LINE = 3,
  OBSTRUCTED_IN_BUILDING = 4,
  OBSTRUCTED_IN_FACTORY = 5

In [3]:
def dB2Veces(potencia):
    return pow(10, potencia/10)

In [4]:
def dBm2pot(dbm):
    return pow(10, dbm/10)*1e-3

In [5]:
def qfunc(x):
    ## Función Q, calculada con la función de error.
    return 0.5-0.5*sp.erf(x/np.sqrt(2))

In [6]:
def get_SNRPerSimbol(snr, bw, sps):
    ## snr: SNR de la señal en el receptor.
    ## bw: Ancho de banda de la señal.
    ## sps: símbolos por segundo. 
    snr = dB2Veces(snr)
    return snr*bw/sps 

In [7]:
def get_SimbolsPerSecond(bw, gi):
    ## bw: Ancho de banda de la señal.
    ## gi: Guard interval(Evita ISI(Interferencia inter-simbólica) por rebotes de la señal).
    periodo = 1/bw
    new_periodo = periodo + gi
    return 1/new_periodo

In [8]:
def get_SER(m, snr, bw, gi):
    ## m: Número de M-QAM
    ## snr: SNR de la señal en el receptor.
    ## bw: Ancho de banda de la señal.
    ## gi: Guard interval(Evita ISI(Interferencia inter-simbólica) por rebotes de la señal).
    sps = get_SimbolsPerSecond(bw, gi)
    gamma = get_SNRPerSimbol(snr, bw, sps)
    #print("Gamma: ", gamma)
    
    return 1-pow((1-(2*(np.sqrt(m)-1)/np.sqrt(m))*qfunc(np.sqrt(3*gamma/(m-1)))), 2)

In [9]:
def get_PER(n, m, snr, bw, gi):
    ## n: N° de bits en un paquete
    ## m: Número de M-QAM
    ## snr: SNR de la señal en el receptor.
    ## bw: Ancho de banda de la señal.
    ## gi: Guard interval(Evita ISI(Interferencia inter-simbólica) por rebotes de la señal).
    ps = get_SER(m, snr, bw, gi)
    ## n_spp: N° de símbolos en un paquete
    n_spp = int(n/np.log2(m))
    #print("SER: ", ps, "\nN° símbolos por paquete: ", n_spp)
    return 1-pow((1-ps), n_spp)

In [10]:
def get_friis_propagation(pt, d, longOnda, ple):
    ## pt: Potencia del transmisor. 
    ## d: Distancia hacia el receptor.
    ## longOnda: Longitud de onda de la portadora.
    ## ple: Path Loss Exponent
    friis = (pt*(longOnda**2)/(((4*np.pi)**2)*((d)**ple)))   
    return 10*np.log10(friis/1e-3)

In [11]:
## return: Potencia de recepcion
def get_2_ray_propagation(pt, d, ht, hr):
    ## pt: Potencia del transmisor. 
    ## d: Distancia hacia el receptor.
    ## ht: Altura de transmisora
    ## hr: Altura de receptora
    L=1
    return 10*np.log10(((pt*(ht**2)*(hr**2))/(d**4)*L)/1e-3)

In [12]:
def get_exponent_by_enviroment(enviroment):
    if(enviroment == Environment.FREE_SPACE):
        return 2
    elif( enviroment == Environment.URBAN_AREA_CELLULAR_RADIO):
        return random.uniform(2.7, 3.5)
    elif( enviroment == Environment.SHADOWED_URBAN_CELLULAR_RADIO):
        return random.uniform(3.0, 5.0)
    elif( enviroment == Environment.IN_BUILDING_LINE):
        return random.uniform(1.6, 1.8)
    elif( enviroment == Environment.OBSTRUCTED_IN_BUILDING):
        return random.uniform(4.0, 6.0)
    elif( enviroment == Environment.OBSTRUCTED_IN_FACTORY):
        return random.uniform(2.0, 3.0)

In [13]:
pr = get_friis_propagation(1,1000, 0.33, 2)
print(f"Potencia en el receptor: {pr}[dBm]")

Potencia en el receptor: -61.61391848288417[dBm]


In [14]:
PER = get_PER(1e3, 16, 10, 1e6, 8e-6)
print(PER)

0.008249787181468404


In [15]:
def get_stdRandNum():
    return np.random.standard_normal()

In [16]:
def isThisPacketWrong(PER):
    area_baja = stats.norm.ppf(PER/2)
    area_alta = stats.norm.ppf(1 - PER/2)
    #print(area_baja, area_alta)
    num = get_stdRandNum()
    if num < area_baja or num > area_alta:
        return True
    else:
        return False

In [17]:
count = 0
max_num = 10000
for i in range(max_num):
    if isThisPacketWrong(PER):
        count +=1
print(f"N° de paquetes con bits errados: {count}\n", f"Probabilidad de PER experimental: {count/max_num}")

N° de paquetes con bits errados: 83
 Probabilidad de PER experimental: 0.0083


In [18]:
def get_RxSNR(S, N):
    S = dB2Veces(S)
    N = dB2Veces(N)
    #print(S, N)
    SNR = S/N
    return 10*np.log10(SNR)

In [19]:
def simulator(max_iter, env, dist, n_bits, bw, gi):
    p_tx = 30 # [dBm] Tx
    #p_noise = -70 # [dBm] Ruido < --- **Justificar**
    #print(dBm2pot(p_tx), p_inRx)
    errs = 0
    count = 0
    while count != max_iter:

        p_noise = -(random.gauss(80, 10))

        exp = get_exponent_by_enviroment(env)
        p_inRx = get_friis_propagation(dBm2pot(p_tx), dist, 0.33, exp)

        Rx_SNR = get_RxSNR(p_inRx, p_noise)
        #print(Rx_SNR)
        PER = get_PER(1e3, 16, Rx_SNR, bw, gi)
        #print(f"PER: {PER}")
        if PER != 0:
             if isThisPacketWrong(PER):
                    errs +=1
        count +=1
    print(f"N° paquetes errados: {errs}, N° total de paquetes {max_iter}, Tasa de trasmisión efectiva: {(max_iter - errs)/max_iter}")

In [22]:
simulator(10000, Environment.FREE_SPACE, 250, 1e3, 1e6, 8e-6)

N° paquetes errados: 128, N° total de paquetes 10000, Tasa de trasmisión efectiva: 0.9872


In [120]:
def simulator(max_iter):
    p_tx = 30 # [dBm] Tx
    mu = 80 # -[dBm]
    sigma = 10 # -[dBm]
    canal_bw = [1e6, 2e6, 4e6, 8e6, 16e6] ## Canales BW
    dist_list = [1, 2, 3, 5, 7, 10,
                 20, 30, 50, 70, 100,
                 200, 300, 500, 700, 1000,
                 2000, 3000, 5000, 7000, 10000] ## Distancias 
    q_am = [2, 4, 16, 64, 256] ## BPSK, QPSK, 16QAM, 64QAM, 256QAM
    gi_list = [4e-6, 8e-6] ## Guard Interval
    n_bits = 1e3
    datos = {}
    errs = 0
    count = 0
    while count != max_iter:
        for env in Environment:
            print(f"Environment: {env}")
            for gi in gi_list:
                print(f"Guard Interval[s]: {gi}")
                for bw in canal_bw:
                    print(f"Channel BW[Hz]: {bw}")
                    for q in q_am:
                        print(f"N° de constelaciónes QAM: {q}")
                        calcular = True
                        for dist in dist_list:
                            if calcular:
                                errs = 0
                                count = 0
                                tasa_efectiva = 1
                                print(f"Distancia al Rx[m]: {dist}")
                                while count != max_iter:
                                    p_noise = -(random.gauss(mu, sigma))

                                    exp = get_exponent_by_enviroment(env)
                                    p_inRx = get_friis_propagation(dBm2pot(p_tx), dist, 0.33, exp)

                                    Rx_SNR = get_RxSNR(p_inRx, p_noise)
                                    #print(Rx_SNR)
                                    PER = get_PER(n_bits, q, Rx_SNR, bw, gi)
                                    #print(f"PER: {PER}")
                                    if PER != 0:
                                         if isThisPacketWrong(PER):
                                                errs +=1

                                    tasa_efectiva = (max_iter - errs)/max_iter
                                    count +=1

                                print(f'''N° paquetes errados: {errs}
                                N° total de paquetes {max_iter}
                                Tasa de trasmisión efectiva: {tasa_efectiva}''')
                            
                            datos[(env, gi, bw, q, dist)] = tasa_efectiva
                            
                            if tasa_efectiva == 0.0:
                                calcular = False
                                
    return datos

In [ ]:
datos = simulator(1e6)

Environment: Environment.FREE_SPACE
Guard Interval[s]: 4e-06
Channel BW[Hz]: 1000000.0
N° de constelaciónes QAM: 2
Distancia al Rx[m]: 1
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 2
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 3
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 5
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 7
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 

N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 7
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 10
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 20
N° paquetes errados: 15
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999985
Distancia al Rx[m]: 30
N° paquetes errados: 72
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999928
Distancia al Rx[m]: 50
N° paquetes errados: 322
                                N° total de paquetes 1000000.0
                               

N° paquetes errados: 1202
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.998798
Distancia al Rx[m]: 30
N° paquetes errados: 3523
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.996477
Distancia al Rx[m]: 50
N° paquetes errados: 12400
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.9876
Distancia al Rx[m]: 70
N° paquetes errados: 25561
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.974439
Distancia al Rx[m]: 100
N° paquetes errados: 49745
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.950255
Distancia al Rx[m]: 200
N° paquetes errados: 146934
                                N° total de paquetes 100000

N° paquetes errados: 75
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999925
Distancia al Rx[m]: 200
N° paquetes errados: 799
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999201
Distancia al Rx[m]: 300
N° paquetes errados: 2512
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.997488
Distancia al Rx[m]: 500
N° paquetes errados: 8883
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.991117
Distancia al Rx[m]: 700
N° paquetes errados: 18794
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.981206
Distancia al Rx[m]: 1000
N° paquetes errados: 38702
                                N° total de paquetes 100000

N° paquetes errados: 136941
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.863059
Distancia al Rx[m]: 700
N° paquetes errados: 210171
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.789829
Distancia al Rx[m]: 1000
N° paquetes errados: 310349
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.689651
Distancia al Rx[m]: 2000
N° paquetes errados: 540841
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.459159
Distancia al Rx[m]: 3000
N° paquetes errados: 674652
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.325348
Distancia al Rx[m]: 5000
N° paquetes errados: 814345
                                N° total de

...

N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 30
N° paquetes errados: 1
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999999
Distancia al Rx[m]: 50
N° paquetes errados: 16
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999984
Distancia al Rx[m]: 70
N° paquetes errados: 34
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999966
Distancia al Rx[m]: 100
N° paquetes errados: 107
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.999893
Distancia al Rx[m]: 200
N° paquetes errados: 1210
                                N° total de paquetes 1000000.0
               

N° paquetes errados: 6984
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.993016
Distancia al Rx[m]: 200
N° paquetes errados: 31312
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.968688
Distancia al Rx[m]: 300
N° paquetes errados: 64998
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.935002
Distancia al Rx[m]: 500
N° paquetes errados: 141559
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.858441
Distancia al Rx[m]: 700
N° paquetes errados: 218087
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.781913
Distancia al Rx[m]: 1000
N° paquetes errados: 318151
                                N° total de paquet

N° paquetes errados: 1640
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.99836
Distancia al Rx[m]: 1000
N° paquetes errados: 4466
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.995534
Distancia al Rx[m]: 2000
N° paquetes errados: 21421
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.978579
Distancia al Rx[m]: 3000
N° paquetes errados: 47103
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.952897
Distancia al Rx[m]: 5000
N° paquetes errados: 108721
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.891279
Distancia al Rx[m]: 7000
N° paquetes errados: 174058
                                N° total de paque

N° paquetes errados: 515797
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.484203
Distancia al Rx[m]: 7000
N° paquetes errados: 629314
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.370686
Distancia al Rx[m]: 10000
N° paquetes errados: 739839
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 0.260161
N° de constelaciónes QAM: 256
Distancia al Rx[m]: 1
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 2
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 3
N° paquetes errados: 0
                                N° total de p

N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 2
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 3
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 5
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 7
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisión efectiva: 1.0
Distancia al Rx[m]: 10
N° paquetes errados: 0
                                N° total de paquetes 1000000.0
                                Tasa de trasmisi

In [ ]:
def plotGraph(data_x, data_y, env, gi, bw, q):
    lenght = len(data_x)
    #x = np.linspace(0, lenght, num=lenght, endpoint=False) ## Equiespaciamos max_In en interval

    # Note that even in the OO-style, we use `.pyplot.figure` to create the figure.
    fig, ax = plt.subplots(figsize=(15,10))  # Create a figure and an axes.
    ax.plot(data_x, data_y, "-bo", label='Goodput')  # Plot some data on the axes.
    ax.set_xlabel('Distancia[m]')  # Add an x-label to the axes.
    ax.set_ylabel('Tasa efectiva paquetes(Goodput)')  # Add a y-label to the axes.
    ax.set_title(f"Goodput. vs Distancia[m]")  # Add a title to the axes.
    ax.legend()  # Add a legend.
    ax.text(6500, 1, f'\n{"#"*len(str(env))}\nAmbiente: {env}\n' +
           f'Guard Interval[us]: {gi/1e-6}\n' +
           f'Bw del canal[MHz]: {bw/1e6}\n' + 
           f'M-QAM: {q}', fontsize=12)
    for i in range(lenght):
        ax.annotate(str(data_x[i]), # this is the text
                     (data_x[i], data_y[i]), # these are the coordinates to position the label
                     textcoords="offset points", # how to position the text
                     xytext=(0,10), # distance from text to points (x,y)
                     ha='center') # horizontal alignment can be left, right or center


In [ ]:
def plotAllGraphs(datos):
    canal_bw = [1e6, 2e6, 4e6, 8e6, 16e6] ## Canales BW
    dist_list = [1, 2, 3, 5, 7, 10,
                 20, 30, 50, 70, 100,
                 200, 300, 500, 700, 1000,
                 2000, 3000, 5000, 7000, 10000] ## Distancias 
    q_am = [2, 4, 16, 64, 256] ## BPSK, QPSK, 16QAM, 64QAM, 256QAM
    gi_list = [4e-6, 8e-6] ## Guard Interval
    data = {}
    tmp = []
    for env in Environment:
        print(env)
        data = {k: v for k, v in datos.items() if k[0]==env}
        for gi in gi_list:
            for bw in canal_bw:
                for q in q_am:
                    tmp = []
                    for dist in dist_list:
                        tmp.append(data[env, gi, bw, q, dist])
                    print(tmp)
                    tmp = np.array(tmp)
                    plotGraph(dist_list, tmp, env, gi, bw, q)
                        

In [ ]:
plotAllGraphs(datos)